# Score

In [ ]:
import sc3nb as scn

In [ ]:
sc = scn.startup()

In [ ]:
from sc3nb import Score, SynthDef

The Score class can be used for non-realtime synthesis. 

* This is done by starting the SuperCollider audio server scsynth in the non-realtime mode.
* The server will read the provided OSC file and render the sound to the specified sound file.
* **Note** that this will require to send all required SynthDefs and Buffers at the beginning. However you can start using the Buffers & SynthDefs immediately after the corresponding OSCMessages as the audio server will handle all messages in the specified order.


The `Score.record_nrt` class method provides an easy interface that generates a OSC file from a dict with timings as keys and lists of `OSCMessages` as values.

In [ ]:
help(Score.record_nrt)

Lets create a simple SynthDef for this demonstration

In [ ]:
synthdef = SynthDef(
    "test",
    r"""{ |out, freq = 440|
            OffsetOut.ar(out,
                SinOsc.ar(freq, 0, 0.2) * Line.kr(1, 0, 0.5, doneAction: Done.freeSelf)
            )
        }""",
)

For creating the messages its recommended to use the Bundler class

In [ ]:
with sc.server.bundler(send_on_exit=False) as bundler:
    synthdef.add()  # Send the test SynthDef
    bundler.add(0.0, "/s_new", ["test", 1003, 0, 0, "freq", 440])
    bundler.add(0.2, "/s_new", ["test", 1000, 0, 0, "freq", 440])
    bundler.add(0.4, "/s_new", ["test", 1001, 0, 0, "freq", 660])
    bundler.add(0.6, "/s_new", ["test", 1002, 0, 0, "freq", 220])
    bundler.add(1, "/c_set", [0, 0])  # The /c_set [0, 0] will close the audio file

The corresponding messages can be seen with

In [ ]:
bundler.messages()

Lets start the non-realtime synthesis

In [ ]:
Score.record_nrt(bundler.messages(), "../media/score.osc", "../media/score.wav", header_format="WAV")

Lets listen to the created audio file with the IPython Audio class that allows to read and play audio files 

In [ ]:
from IPython.display import Audio

In [ ]:
Audio("../media/score.wav")

In [ ]:
sc.exit()